In [160]:
import gzip
import pandas as pd
import numpy as np
import pickle
import requests
import logging
import time
import random
import re
import json

f = open("token_TMDB.txt","r")
token = f.read()
headers = {"accept": "application/json", "Authorization": f"Bearer {token}"}

In [161]:
movies = pickle.load(open('data.pkl', 'rb'))

In [162]:
movies 

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,keywords,overview,production_countries
28768,tt0029284,My Favorite Wife,My Favorite Wife,1940,88,"{'keywords': [{'id': 931, 'name': 'jealousy'},...",Years after she was presumed dead in a shipwre...,"[{'iso_3166_1': 'US', 'name': 'United States o..."
30799,tt0031359,Gaslight,Gaslight,1940,84,"{'keywords': [{'id': 6038, 'name': 'marriage'}...",Twenty years removed from Alice Barlow's murde...,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]"
31405,tt0031976,The Stars Look Down,The Stars Look Down,1940,110,"{'keywords': [{'id': 6346, 'name': 'trade unio...",Davey Fenwick leaves his mining village on a u...,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]"
31600,tt0032179,21 Days Together,21 Days,1940,72,"{'keywords': [{'id': 171989, 'name': 'wrongful...",After Larry Darrent accidentally kills his lov...,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]"
31602,tt0032181,Abe Lincoln in Illinois,Abe Lincoln in Illinois,1940,110,"{'keywords': [{'id': 5565, 'name': 'biography'...",Abe Lincoln in Illinois is a 1940 biographical...,"[{'iso_3166_1': 'US', 'name': 'United States o..."
...,...,...,...,...,...,...,...,...
10698589,tt9907782,The Cursed,Eight for Silver,2021,111,"{'keywords': [{'id': 394, 'name': 'gypsy'}, {'...","In the late 19th century, a brutal land baron ...","[{'iso_3166_1': 'US', 'name': 'United States o..."
10698877,tt9908390,Le lion,Le lion,2020,95,"{'keywords': [{'id': 5265, 'name': 'espionage'}]}",A psychiatric hospital patient pretends to be ...,"[{'iso_3166_1': 'FR', 'name': 'France'}]"
10700132,tt9911196,The Marriage Escape,De beentjes van Sint-Hildegard,2020,103,{'keywords': []},Jan has been married to Gedda for 35 years. Ge...,"[{'iso_3166_1': 'NL', 'name': 'Netherlands'}]"
10702488,tt9916270,Il talento del calabrone,Il talento del calabrone,2020,84,{'keywords': []},"Dj Steph is a young radio deejay on the rise, ...","[{'iso_3166_1': 'IT', 'name': 'Italy'}]"


In [163]:
def get_keywords(row):
    if isinstance(row, list):  # Check if it's already a list
        kws = [kw['name'] for kw in row]
        return ' '.join(kws)
    elif isinstance(row, dict):  # If it's a dictionary, directly access 'keywords' key
        kws = [kw['name'] for kw in row['keywords']]
        return ' '.join(kws)
    else:
        return ''

In [164]:
def get_countries(data, separator=' '):
    if data is None:
        return ''
    
    top_countries = ['US', 'GB', 'CN', 'FR', 'JP', 'DE', 'KR', 'CA', 'AU', 'IN']
    
    countries = []
    for country in data:
        name = format_name(country['name'])
        if name not in top_countries:
            countries.append(name)
        
    return separator.join(countries)

In [165]:
def format_name(name):
    if name is None:
        return None
    name = re.sub(r'[^\w\s]', '', name)
    parts = name.split()
    return '_'.join(parts)

In [166]:
def remove_punc(text):
    if text is None:
        return None
    elif isinstance(text, int):
        return text
    else:
        text = re.sub(r'[^\w\s]', ' ', str(text))  # Convert to string before applying regex
        parts = text.split()
    return ' '.join(parts)


In [167]:
movies['production_countries'] = movies['production_countries'].apply(get_countries)
movies['keywords'] = movies['keywords'].apply(get_keywords)

In [168]:
movies = movies.applymap(remove_punc)

/var/folders/_8/f5zjdbxn0vs8y_m3qv7n_8800000gn/T/ipykernel_12452/2664865618.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  movies = movies.applymap(remove_punc)


In [169]:
genres = pickle.load(open('genres.pkl', 'rb'))
ratings = pickle.load(open('ratings.pkl', 'rb'))

In [170]:
movies = pd.merge(movies, ratings, on='tconst', how='inner')

In [171]:
movies

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,keywords,overview,production_countries,averageRating,numVotes
0,tt0029284,My Favorite Wife,My Favorite Wife,1940,88,jealousy judge shipwreck marriage bigamy confu...,Years after she was presumed dead in a shipwre...,United_States_of_America,7.3,11501
1,tt0031359,Gaslight,Gaslight,1940,84,marriage murder driven mad rubies,Twenty years removed from Alice Barlow s murde...,United_Kingdom,7.3,5264
2,tt0031976,The Stars Look Down,The Stars Look Down,1940,110,trade union miner social commentary labor union,Davey Fenwick leaves his mining village on a u...,United_Kingdom,7.0,1178
3,tt0032179,21 Days Together,21 Days,1940,72,wrongful conviction,After Larry Darrent accidentally kills his lov...,United_Kingdom,6.1,1276
4,tt0032181,Abe Lincoln in Illinois,Abe Lincoln in Illinois,1940,110,biography historical figure abraham lincoln,Abe Lincoln in Illinois is a 1940 biographical...,United_States_of_America,7.3,2052
...,...,...,...,...,...,...,...,...,...,...
32175,tt9907782,The Cursed,Eight for Silver,2021,111,gypsy world war i gore flashback werewolf crea...,In the late 19th century a brutal land baron s...,United_States_of_America,6.2,16278
32176,tt9908390,Le lion,Le lion,2020,95,espionage,A psychiatric hospital patient pretends to be ...,France,5.5,1407
32177,tt9911196,The Marriage Escape,De beentjes van Sint Hildegard,2020,103,,Jan has been married to Gedda for 35 years Ged...,Netherlands,7.4,3321
32178,tt9916270,Il talento del calabrone,Il talento del calabrone,2020,84,,Dj Steph is a young radio deejay on the rise w...,Italy,5.8,1480


In [172]:
movies = pd.merge(movies, genres.groupby('tconst')['genre'].agg(lambda x: ' '.join(x)), on='tconst', how='inner')

In [173]:
movies.dropna(inplace=True)

In [174]:
relations = pickle.load(open('cast_crew.pkl', 'rb'))
people = pickle.load(open('people.pkl', 'rb'))

In [175]:
relations = pd.merge(relations, people[['nconst', 'primaryName']], on='nconst', how='inner')

In [176]:
cast_crew = relations.pivot_table(index='tconst', columns='profession', values='primaryName', aggfunc=lambda x: ','.join(x)).reset_index()

In [177]:
movies = pd.merge(movies, cast_crew, on='tconst', how='left')


In [178]:
movies

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,keywords,overview,production_countries,averageRating,numVotes,genre,actor,actress,director,writers
0,tt0029284,My Favorite Wife,My Favorite Wife,1940,88,jealousy judge shipwreck marriage bigamy confu...,Years after she was presumed dead in a shipwre...,United_States_of_America,7.3,11501,Comedy Romance,"Cary Grant,Randolph Scott,Scotty Beckett,Donal...","Irene Dunne,Gail Patrick,Ann Shoemaker,Mary Lo...",Garson Kanin,"Bella Spewack,Garson Kanin,Sam Spewack,John Mc..."
1,tt0031359,Gaslight,Gaslight,1940,84,marriage murder driven mad rubies,Twenty years removed from Alice Barlow s murde...,United_Kingdom,7.3,5264,Mystery Thriller,"Anton Walbrook,Frank Pettingell,Robert Newton,...","Diana Wynyard,Cathleen Cordell,Minnie Rayner,M...",Thorold Dickinson,"Bridget Boland,A.R. Rawlinson,Patrick Hamilton"
2,tt0031976,The Stars Look Down,The Stars Look Down,1940,110,trade union miner social commentary labor union,Davey Fenwick leaves his mining village on a u...,United_Kingdom,7.0,1178,Drama,"Michael Redgrave,Edward Rigby,Emlyn Williams,A...","Margaret Lockwood,Nancy Price,Linden Travers",Carol Reed,"J.B. Williams,A.J. Cronin,A. Coppel"
3,tt0032179,21 Days Together,21 Days,1940,72,wrongful conviction,After Larry Darrent accidentally kills his lov...,United_Kingdom,6.1,1276,Crime Drama Romance,"Leslie Banks,Laurence Olivier,Francis L. Sulli...","Vivien Leigh,Elliott Mason",Basil Dean,"Basil Dean,Graham Greene,John Galsworthy"
4,tt0032181,Abe Lincoln in Illinois,Abe Lincoln in Illinois,1940,110,biography historical figure abraham lincoln,Abe Lincoln in Illinois is a 1940 biographical...,United_States_of_America,7.3,2052,Biography Drama History,"Raymond Massey,Gene Lockhart,Minor Watson,Alan...","Ruth Gordon,Mary Howard,Dorothy Tree",John Cromwell,"Grover Jones,Robert E. Sherwood"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31959,tt9907782,The Cursed,Eight for Silver,2021,111,gypsy world war i gore flashback werewolf crea...,In the late 19th century a brutal land baron s...,United_States_of_America,6.2,16278,Fantasy Horror Mystery,"Boyd Holbrook,Alistair Petrie,Nigel Betts,Stua...","Kelly Reilly,Roxane Duran,Amelia Crouch",Sean Ellis,Sean Ellis
31960,tt9908390,Le lion,Le lion,2020,95,espionage,A psychiatric hospital patient pretends to be ...,France,5.5,1407,Comedy,"Dany Boon,Philippe Katerine,Samuel Jouy,Benoît...","Anne Serra,Sophie Verbeeck,Carole Brana",Ludovic Colbeau-Justin,"Alexandre Coquelle,Matthieu Le Naour"
31961,tt9911196,The Marriage Escape,De beentjes van Sint Hildegard,2020,103,,Jan has been married to Gedda for 35 years Ged...,Netherlands,7.4,3321,Comedy Drama,"Herman Finkers,Ferdi Stofmeel,Stef Assen,Jan R...","Johanna ter Steege,Leonie ter Braak,Aniek Stok...",Johan Nijenhuis,"Herman Finkers,Maarten Lebens,Mirka Zlatníková..."
31962,tt9916270,Il talento del calabrone,Il talento del calabrone,2020,84,,Dj Steph is a young radio deejay on the rise w...,Italy,5.8,1480,Thriller,"Sergio Castellitto,Lorenzo Richelmy,David Coco...","Anna Foglietta,Marina Occhionero,Cristina Mari...",Giacomo Cimini,"Giacomo Cimini,Lorenzo Collalti"


In [179]:
movies['cast'] = movies['actor'].str.cat(movies['actress'], sep=',')

In [180]:
movies.drop(columns=['actor','actress'],inplace=True)

In [181]:
movies.rename(columns={
    'tconst': 'id',
    'primaryTitle': 'primary_title',
    'originalTitle': 'original_title',
    'startYear': 'release_date',
    'runtimeMinutes': 'runtime',
    'averageRating': 'average_rating',
    'numVotes': 'vote_count',
    'genre': 'genres'
}, inplace=True)

In [182]:
movies

,id,primary_title,original_title,release_date,runtime,keywords,overview,production_countries,average_rating,vote_count,genres,director,writers,cast
0,tt0029284,My Favorite Wife,My Favorite Wife,1940,88,jealousy judge shipwreck marriage bigamy confu...,Years after she was presumed dead in a shipwre...,United_States_of_America,7.3,11501,Comedy Romance,Garson Kanin,"Bella Spewack,Garson Kanin,Sam Spewack,John Mc...","Cary Grant,Randolph Scott,Scotty Beckett,Donal..."
1,tt0031359,Gaslight,Gaslight,1940,84,marriage murder driven mad rubies,Twenty years removed from Alice Barlow s murde...,United_Kingdom,7.3,5264,Mystery Thriller,Thorold Dickinson,"Bridget Boland,A.R. Rawlinson,Patrick Hamilton","Anton Walbrook,Frank Pettingell,Robert Newton,..."
2,tt0031976,The Stars Look Down,The Stars Look Down,1940,110,trade union miner social commentary labor union,Davey Fenwick leaves his mining village on a u...,United_Kingdom,7.0,1178,Drama,Carol Reed,"J.B. Williams,A.J. Cronin,A. Coppel","Michael Redgrave,Edward Rigby,Emlyn Williams,A..."
3,tt0032179,21 Days Together,21 Days,1940,72,wrongful conviction,After Larry Darrent accidentally kills his lov...,United_Kingdom,6.1,1276,Crime Drama Romance,Basil Dean,"Basil Dean,Graham Greene,John Galsworthy","Leslie Banks,Laurence Olivier,Francis L. Sulli..."
4,tt0032181,Abe Lincoln in Illinois,Abe Lincoln in Illinois,1940,110,biography historical figure abraham lincoln,Abe Lincoln in Illinois is a 1940 biographical...,United_States_of_America,7.3,2052,Biography Drama History,John Cromwell,"Grover Jones,Robert E. Sherwood","Raymond Massey,Gene Lockhart,Minor Watson,Alan..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31959,tt9907782,The Cursed,Eight for Silver,2021,111,gypsy world war i gore flashback werewolf crea...,In the late 19th century a brutal land baron s...,United_States_of_America,6.2,16278,Fantasy Horror Mystery,Sean Ellis,Sean Ellis,"Boyd Holbrook,Alistair Petrie,Nigel Betts,Stua..."
31960,tt9908390,Le lion,Le lion,2020,95,espionage,A psychiatric hospital patient pretends to be ...,France,5.5,1407,Comedy,Ludovic Colbeau-Justin,"Alexandre Coquelle,Matthieu Le Naour","Dany Boon,Philippe Katerine,Samuel Jouy,Benoît..."
31961,tt9911196,The Marriage Escape,De beentjes van Sint Hildegard,2020,103,,Jan has been married to Gedda for 35 years Ged...,Netherlands,7.4,3321,Comedy Drama,Johan Nijenhuis,"Herman Finkers,Maarten Lebens,Mirka Zlatníková...","Herman Finkers,Ferdi Stofmeel,Stef Assen,Jan R..."
31962,tt9916270,Il talento del calabrone,Il talento del calabrone,2020,84,,Dj Steph is a young radio deejay on the rise w...,Italy,5.8,1480,Thriller,Giacomo Cimini,"Giacomo Cimini,Lorenzo Collalti","Sergio Castellitto,Lorenzo Richelmy,David Coco..."


In [183]:
movies.to_pickle('merged.pkl')

In [184]:
movies.columns

Index(['id', 'primary_title', 'original_title', 'release_date', 'runtime',
       'keywords', 'overview', 'production_countries', 'average_rating',
       'vote_count', 'genres', 'director', 'writers', 'cast'],
      dtype='object')

In [ ]:
data = df[['id','genres','original_title','keywords','overview',
           'popularity','release_date','runtime','vote_average','vote_count',
           'cast','crew','production_countries','production_companies']]
data.rename(columns={'original_title': 'title'}, inplace=True)
data.to_csv('mini_data.csv', index=False)